In [49]:
from __future__ import print_function
import numpy as np
import cv2
from keras.models import load_model
from resunet import ResUnet
import time
from unet2 import *
from mydatabinary import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, CSVLogger
import keras.backend as K
K.set_image_data_format('channels_last')
import os,cv2, sys, math , csv
from keras import optimizers
from pathlib import Path
from m_resunet import ResUnetPlusPlus
print(tf.__version__)


2.6.0


In [50]:
def fscore1(y_true,y_pred):

    y_true = y_true[:,:,:,1]
    y_pred = y_pred[:,:,:,1]
    y_true = K.round(K.flatten(y_true))

    y_pred = K.round(K.flatten(y_pred)) 
    true_positives = K.sum(y_true * y_pred)
    predicted_positives = K.sum(y_pred)
    possible_positives = K.sum(y_true)
    precision = true_positives / (predicted_positives+ K.epsilon())   
    recall = true_positives / (possible_positives+ K.epsilon())  
    f_score = 2*precision*recall /(precision+recall+ K.epsilon())
    return f_score

def fscore(y_true,y_pred):
  f_score=0
  for i in range(4):
    y_true1=y_true[i,:,:,:]
    c=tf.math.argmax(tf.raw_ops.UniqueV2(x=tf.reshape(y_true1,[-1,34]),axis=[0])[1])
    y_true = y_true[i,:,:,c]
    y_pred = y_pred[i,:,:,c]
    y_true = K.round(K.flatten(y_true))

    y_pred = K.round(K.flatten(y_pred)) 
    true_positives = K.sum(y_true * y_pred)
    predicted_positives = K.sum(y_pred)
    possible_positives = K.sum(y_true)
    precision = true_positives / (predicted_positives+ K.epsilon())   
    recall = true_positives / (possible_positives+ K.epsilon())  
    f_score = f_score + (2*precision*recall /(precision+recall+ K.epsilon()))
  return f_score/4
  
def weighted_dice_loss(weights):
  def dice_loss(y_true, y_pred):
    Kweights = K.constant(weights)
    y_true = tf.cast(y_true, tf.float32)
    #y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - numerator*Kweights / denominator   
  return dice_loss

def custom_loss(y_true,y_pred):
  loss = -K.mean(weights*y_true*K.log(y_pred+K.epsilon()))
  return loss

def weighted_categorical_crossentropy(weights):
    # weights = [0.9,0.05,0.04,0.01]
    def wcce(y_true, y_pred):
        Kweights = K.constant(weights)
        #if not K.is_tensor(y_pred): y_pred = K.constant(y_pred)
        y_true = K.cast(y_true, y_pred.dtype)
        return K.categorical_crossentropy(y_true, y_pred) * K.sum(y_true * Kweights, axis=-1)
    return wcce

In [51]:
# img1=cv2.imread("D:\\bac_clas\\data\\diabs\\Acinetobacter.baumanii\\masked\\Acinetobacter.baumanii_0004_gt.tif")
# img=cv2.imread("D:\\bac_clas\\data\\diabs\\Actinomyces.israeli\\masked\\Actinomyces.israeli_0002_gt.tif")
# img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
# img1=cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
# img111=cv2.imread("D:\\bac_clas\\data\\diabs\\Acinetobacter.baumanii\\masked\\Acinetobacter.baumanii_0004_gt.tif")
# img111=cv2.cvtColor(img111,cv2.COLOR_BGR2RGB)
# img=np.uint8(np.stack((img1,img1,img,img)))[:,0:512,0:512,:]
# img1=np.uint8(np.stack((img1,img1,img1,img1)))[:,0:512,0:512,:]
# img22=one_hot(img)
# img11=one_hot(img1)
# img111=one_hot(img111)
# # tf.print(fscore(img11,img11))
# c=np.unique(img11[1].reshape((-1,34)),axis=0)[1]


In [52]:
# c=np.unique(img11[1],axis=0)[1]
# imgt=tf.convert_to_tensor(
#     img11[1], dtype=None, dtype_hint=None, name=None
# )
# tf.raw_ops.UniqueV2(x=tf.reshape(imgt,[-1,34]),axis=[0])

In [53]:
# for i in range(10):


In [54]:
path = os.path.dirname(os.getcwd())
path = Path(path)
magnification = "100x"

val_name = path/"data/validationcr"
train_name = path/"data/traincr/"
csv_name = path/"modeloutputs/results/resUnetpp_weighted_cce_loss_DiBaS_dataset_both_gp_gn1.csv"
weights_folder =path/"modeloutputs/weights/weights_resUnetpp_weighted_cce_loss_DiBaS_dataset_both_gp_gn"
model_name_morph = path/"modeloutputs/modelh5"

batch_size = 4 # No. of images in a batch
size = 512


data_gen_train = dict(rotation_range=10,
                    horizontal_flip=True,
                    width_shift_range=5,
                    height_shift_range=5,
	   	              vertical_flip=True,
                    fill_mode='nearest')


In [55]:
num_images = num_of_images(train_name)
num_images_v=num_of_images(val_name)

Found 8300 images belonging to 1 classes.
Found 1704 images belonging to 1 classes.


In [58]:
train = dataGenerator(batch_size,train_name,data_gen_train,size)

val = dataGeneratorv(batch_size,val_name,size)

loss_fn = 'categorical_crossentropy'
batch_steps = np.ceil(num_images / batch_size)
print(batch_steps)
rms = tf.keras.optimizers.RMSprop(learning_rate=1e-5)
opt = rms

if os.path.isdir(weights_folder) is not True:
    os.makedirs(weights_folder)


checkpointer = ModelCheckpoint(filepath='%s/weights.bac.{epoch:03d}_resunetpp.hdf5'%weights_folder,save_weights_only= True,save_best_only=True,monitor='val_loss',mode='min')
start = time.time()
checkpoint_path = str(model_name_morph)
checkpoint = False

if checkpoint:
    arch = ResUnetPlusPlus(input_size=size,no_classes=34)
    model = arch.build_model()
    #model = load_model(checkpoint_path, custom_objects={'fscore': fscore,'loss_fn':loss_fn})
    initial_epoch = 15
else:
    arch = ResUnetPlusPlus(input_size=size,no_classes=34)

    model = arch.build_model()
    # model.load_weights("%s/weights.bac.018_resunetpp.hdf5"%str(weights_folder)) # To continue from a particular epoch
    initial_epoch = 0

final_epoch = 1
model.compile(optimizer = opt, loss = loss_fn, metrics =['accuracy',fscore])
model.fit(train, steps_per_epoch=batch_steps,epochs=final_epoch, verbose=1, validation_data=val, validation_steps=batch_steps,  callbacks=[checkpointer],shuffle=True, initial_epoch=initial_epoch)
model.save(str(model_name_morph))
end = time.time()
print(f"time taken for training {final_epoch - initial_epoch} is {end - start} seconds")


2075.0
KerasTensor(type_spec=TensorSpec(shape=(None, 512, 512, 34), dtype=tf.float32, name=None), name='activation_68/Softmax:0', description="created by layer 'activation_68'")
Found 8300 images belonging to 1 classes.
Found 8300 images belonging to 1 classes.
(4, 512, 512, 3)


ValueError: in user code:

    c:\Users\cheth\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\cheth\AppData\Local\Temp\ipykernel_10108\265189632.py:19 fscore  *
        y_true1=y_true[i,:,:,:]
    c:\Users\cheth\anaconda3\envs\tf\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
        return target(*args, **kwargs)
    c:\Users\cheth\anaconda3\envs\tf\lib\site-packages\tensorflow\python\ops\array_ops.py:1041 _slice_helper
        return strided_slice(
    c:\Users\cheth\anaconda3\envs\tf\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
        return target(*args, **kwargs)
    c:\Users\cheth\anaconda3\envs\tf\lib\site-packages\tensorflow\python\ops\array_ops.py:1214 strided_slice
        op = gen_array_ops.strided_slice(
    c:\Users\cheth\anaconda3\envs\tf\lib\site-packages\tensorflow\python\ops\gen_array_ops.py:10537 strided_slice
        _, _, _op, _outputs 

In [ ]:

l=os.listdir(train_name/'masks')
l.sort()
for i in l:
    img=cv.imread(os.path.join(train_name/'masks',i))
    if len(np.unique(img)) > 5:
        print('problem',i,np.unique(img))



In [ ]:
train = dataGenerator(batch_size,train_name,data_gen_train,size)
# x,y=next(train)
# print(y.shape)
# y=y.reshape((y.shape[0]*y.shape[1]*y.shape[2]),y.shape[3])
# print(np.unique(y,axis=0))
j=0
for i in train:

#     if j>-1:
#         print(i[1][1][100][100])
#         print(np.unique(i[1][1].reshape(512*512,34),axis=0,return_counts=True))

#     if j==8:
#             break
#     j=j+1
      for a in range(4):
        for b in range(512):
            for c in range(512):
               if  np.sum((i[1][a,b,c]))==0:
                   print(i[1][a,b,c])
                   print('problem',a,b,c)
   
      j=j+1

In [ ]:
train = dataGenerator(batch_size,train_name,data_gen_train,size)
#train = dataGeneratorv(batch_size,train_name,size)
for i in train:
    print(i[1].shape)
    break

In [ ]:
for x,y in train:
    y=y.reshape((y.shape[0]*y.shape[1]*y.shape[2]),y.shape[3])
    if len(np.unique(y,axis=0))>2:
        print('problem')


In [ ]:

im=os.listdir(train_name/'images')
p=train_name/'masks'
mask=os.listdir(train_name/'masks')

# im.sort()
# mask.sort()
mask=[i[:-7]+'.tif' for i in mask]
# mask.sort()
print(mask[:10])
im[:10]




In [ ]:
import os 
from pathlib import Path
path = os.path.dirname(os.getcwd())
path = Path(path)
magnification = "100x"

val_name = path/"data/validationcr"
train_name = path/"data/traincr/"
csv_name = path/"modeloutputs/results/resUnetpp_weighted_cce_loss_DiBaS_dataset_both_gp_gn1.csv"
weights_folder =path/"modeloutputs/weights/weights_resUnetpp_weighted_cce_loss_DiBaS_dataset_both_gp_gn"
model_name_morph = path/"modeloutputs/modelh5"

p=val_name/'masks'
mask=os.listdir(p)

for i in mask:
    os.rename(p/i,p/('mask_'+i[:-7]+'.tif'))
